# KTTC - AI Translation Quality Assurance Demo

[![GitHub](https://img.shields.io/badge/GitHub-kttc--ai%2Fkttc-blue?logo=github)](https://github.com/kttc-ai/kttc)
[![PyPI](https://img.shields.io/pypi/v/kttc)](https://pypi.org/project/kttc/)
[![Documentation](https://img.shields.io/badge/docs-available-brightgreen)](https://github.com/kttc-ai/kttc/tree/main/docs)

**Welcome!** This notebook demonstrates KTTC's AI-powered translation quality assurance system.

**What is KTTC?**
- Multi-agent QA system that evaluates translation quality using MQM (Multidimensional Quality Metrics)
- Supports English, Russian, Chinese, Hindi, and Persian
- Works with OpenAI, Anthropic, GigaChat, and YandexGPT

**Two modes:**
1. **Demo Mode** - No API key needed, simulated responses
2. **Full Mode** - Use your OpenAI/Anthropic API key for real evaluation

## 📦 Installation

Install KTTC from PyPI (takes ~2 minutes):

In [ ]:
!pip install -q kttc

## Choose Mode

**Option A: Demo Mode** (no API key needed)
- Uses simulated responses
- Perfect for testing the workflow

**Option B: Full Mode** (requires API key)
- Real AI evaluation
- Get your key: [OpenAI](https://platform.openai.com/api-keys) or [Anthropic](https://console.anthropic.com/)

In [ ]:
import os
from getpass import getpass

# Choose mode
USE_DEMO_MODE = True  # Set to False to use your API key

if not USE_DEMO_MODE:
    print("Choose your LLM provider:")
    print("1. OpenAI (GPT-4)")
    print("2. Anthropic (Claude)")

    provider = input("Enter 1 or 2: ").strip()

    if provider == "1":
        api_key = getpass("Enter your OpenAI API key: ")
        os.environ['KTTC_OPENAI_API_KEY'] = api_key
        print("✅ OpenAI API key set!")
    elif provider == "2":
        api_key = getpass("Enter your Anthropic API key: ")
        os.environ['KTTC_ANTHROPIC_API_KEY'] = api_key
        print("✅ Anthropic API key set!")
    else:
        print("❌ Invalid choice, using demo mode")
        USE_DEMO_MODE = True

if USE_DEMO_MODE:
    print("🎭 Demo mode enabled - no API calls will be made")

## Example 1: Single Translation Check

Check the quality of a single translation (English → Russian)

In [ ]:
# Create sample files
with open('source.txt', 'w', encoding='utf-8') as f:
    f.write("""Artificial intelligence is transforming the way we work and live.
Machine learning algorithms can now process vast amounts of data in seconds.
This technology opens up new possibilities for innovation and creativity.""")

# Good translation
with open('translation_good.txt', 'w', encoding='utf-8') as f:
    f.write("""Искусственный интеллект трансформирует то, как мы работаем и живём.
Алгоритмы машинного обучения теперь могут обрабатывать огромные объёмы данных за секунды.
Эта технология открывает новые возможности для инноваций и креативности.""")

# Bad translation (with errors)
with open('translation_bad.txt', 'w', encoding='utf-8') as f:
    f.write("""Искусственный интеллект трансформирует как мы работать и жить.
Машина обучение алгоритмы может обработать большой данные в секунды.
Это технология открывает возможности для инновация.""")

print("✅ Sample files created!")

In [ ]:
import os

# Check good translation
demo_flag = "--demo" if USE_DEMO_MODE else ""
os.system(f'kttc check source.txt translation_good.txt --source-lang en --target-lang ru {demo_flag}')

In [ ]:
# Check bad translation
os.system(f'kttc check source.txt translation_bad.txt --source-lang en --target-lang ru {demo_flag}')

## Example 2: Compare Multiple Translations

Compare multiple translations side-by-side to find the best one

In [ ]:
# Compare good vs bad translation
os.system(f'kttc compare --source source.txt --translations translation_good.txt translation_bad.txt --source-lang en --target-lang ru {demo_flag}')

## Example 3: Batch Processing

Process multiple translation pairs from a CSV file

In [ ]:
import csv

# Create batch CSV file
batch_data = [
    ["source", "translation", "source_lang", "target_lang"],
    ["Hello, world!", "Привет, мир!", "en", "ru"],
    ["Good morning!", "Доброе утро!", "en", "ru"],
    ["Thank you very much.", "Большое спасибо.", "en", "ru"],
    ["How are you?", "Как дела?", "en", "ru"],
]

with open('batch.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(batch_data)

print("✅ Batch CSV created!")
os.system('cat batch.csv')

In [ ]:
# Process batch
os.system(f'kttc batch --file batch.csv --format json --output batch_results.json {demo_flag}')

# Show results
print("\n📄 Results saved to batch_results.json")
os.system('cat batch_results.json')

## Example 4: Python API

Use KTTC programmatically in Python code

In [ ]:
from kttc.agents import AgentOrchestrator
from kttc.core import TranslationTask

if USE_DEMO_MODE:
    from kttc.cli.demo import DemoLLMProvider
    llm = DemoLLMProvider()
else:
    # Use real LLM provider
    if 'KTTC_OPENAI_API_KEY' in os.environ:
        from kttc.llm import OpenAIProvider
        llm = OpenAIProvider(api_key=os.environ['KTTC_OPENAI_API_KEY'])
    else:
        from kttc.llm import AnthropicProvider
        llm = AnthropicProvider(api_key=os.environ['KTTC_ANTHROPIC_API_KEY'])

async def check_quality():
    orchestrator = AgentOrchestrator(llm)

    task = TranslationTask(
        source_text="The future of AI is here.",
        translation="Будущее ИИ здесь.",
        source_lang="en",
        target_lang="ru",
    )

    print("🔍 Evaluating translation quality...")
    report = await orchestrator.evaluate(task)

    print(f"\n📊 MQM Score: {report.mqm_score:.2f}")
    print(f"✅ Status: {report.status.upper()}")
    print(f"⚠️  Errors found: {len(report.errors)}")

    if report.errors:
        print("\n🔍 Issues:")
        for i, error in enumerate(report.errors, 1):
            print(f"  {i}. [{error.severity.name}] {error.category}")
            print(f"     {error.description}")

# Run async function
await check_quality()

## Next Steps

**Install locally:**
```bash
pip install kttc
export KTTC_OPENAI_API_KEY="your-key"
kttc check source.txt translation.txt
```

**Resources:**
- 📖 [Documentation](https://github.com/kttc-ai/kttc/tree/main/docs)
- 💻 [GitHub Repository](https://github.com/kttc-ai/kttc)
- 🐛 [Report Issues](https://github.com/kttc-ai/kttc/issues)
- 💬 [Discussions](https://github.com/kttc-ai/kttc/discussions)

**Language Support:**
- English (LanguageTool: 5,000+ grammar rules)
- Russian (Mawo: morphology, cases, grammar)
- Chinese (HanLP: measure words, particles)
- Hindi (Indic NLP + Stanza + Spello)
- Persian (DadmaTools: spaCy-based)

---

**Enjoyed this demo?** ⭐ Star us on [GitHub](https://github.com/kttc-ai/kttc)!